生成结点信息，供deepwalk读取

node1 node2

node1 node3

node1 node4

node1 node5

In [34]:
# 导入包
import pandas as pd
from tqdm import tqdm

generate='./data/generate/'
path='./data/rec_data/'

# 读取数据，content和entity
ids=[]
data1=[]
data2=[]
entity=[]
with open(path+'recommend_content_entity_0317.txt','r') as fileTrainRaw:
  for line in tqdm(fileTrainRaw):
    tmps=eval(line)
    id=tmps.get('id')
    if id in ids:
        continue
    else:
        ids.append(id)
    tmps=tmps.get('entity')
    n=len(tmps)
    if n<2:
      continue
    for i in range(n):
      if i==0:
        data1.append(tmps[i].replace(" ",""))
        data2.append(tmps[i+1].replace(" ",""))
      elif i==n-1:
        data1.append(tmps[i].replace(" ",""))
        data2.append(tmps[i-1].replace(" ",""))
      else:
        data1.append(tmps[i].replace(" ",""))
        data2.append(tmps[i-1].replace(" ",""))
        data1.append(tmps[i].replace(" ",""))
        data2.append(tmps[i + 1].replace(" ",""))
with open(path+'recommend_content_entity_train.txt','r') as fileTrainRaw:
  for line in tqdm(fileTrainRaw):
    tmps=eval(line)
    id=tmps.get('id')
    if id in ids:
        continue
    else:
        ids.append(id)
    tmps=tmps.get('entity')
    n=len(tmps)
    if n<2:
      continue
    for i in range(n):
      if i==0:
        data1.append(tmps[i].replace(" ",""))
        data2.append(tmps[i+1].replace(" ",""))
      elif i==n-1:
        data1.append(tmps[i].replace(" ",""))
        data2.append(tmps[i-1].replace(" ",""))
      else:
        data1.append(tmps[i].replace(" ",""))
        data2.append(tmps[i-1].replace(" ",""))
        data1.append(tmps[i].replace(" ",""))
        data2.append(tmps[i + 1].replace(" ",""))
with open(path+'recommend_content_entity_test.txt','r') as fileTrainRaw:
  for line in tqdm(fileTrainRaw):
    tmps=eval(line)
    id=tmps.get('id')
    if id in ids:
        continue
    else:
        ids.append(id)
    tmps=tmps.get('entity')
    n=len(tmps)
    if n<2:
      continue
    for i in range(n):
      if i==0:
        data1.append(tmps[i].replace(" ",""))
        data2.append(tmps[i+1].replace(" ",""))
      elif i==n-1:
        data1.append(tmps[i].replace(" ",""))
        data2.append(tmps[i-1].replace(" ",""))
      else:
        data1.append(tmps[i].replace(" ",""))
        data2.append(tmps[i-1].replace(" ",""))
        data1.append(tmps[i].replace(" ",""))
        data2.append(tmps[i + 1].replace(" ",""))

    
cols={
  'col1':data1,
  'col2':data2
}
data=pd.DataFrame(cols)

data=data[['col1','col2']].value_counts()
data=data.to_frame().reset_index()
data.rename(columns={0:'weight'},inplace=True)

# data.drop_duplicates(inplace=True)
data.to_csv(generate+'entitys.txt',sep='\t',index=False,header=False)

146791it [03:15, 751.05it/s]
1446it [00:03, 428.15it/s]
1535it [00:03, 403.30it/s]


In [2]:
import networkx as nx
import random
from tqdm import tqdm

generate='./data/generate/'
path='./data/rec_data/'
# create_using=nx.DiGraph() 表示构造的是有向图
G = nx.read_edgelist(generate+'entitys.txt', create_using=nx.DiGraph())

# 从 start_node 开始随机游走
def deepwalk_walk(walk_length, start_node):
    walk = [start_node]
    while len(walk) < walk_length:
        cur = walk[-1]
        cur_nbrs = list(G.neighbors(cur))
        if len(cur_nbrs) > 0:
            walk.append(random.choice(cur_nbrs))
        else:
            break
    return walk

# 产生随机游走序列
def _simulate_walks(nodes, num_walks, walk_length):
    walks = []
    for _ in tqdm(range(num_walks)):
        random.shuffle(nodes)
        for v in nodes:
            walks.append(deepwalk_walk(walk_length=walk_length, start_node=v))
    return walks

# 得到所有节点
nodes = list(G.nodes())
# 得到序列
walks = _simulate_walks(nodes, num_walks=80, walk_length=10)


from gensim.models import Word2Vec
# 默认嵌入到100维
w2v_model = Word2Vec(walks,sg=1,hs=1,vector_size=64)
# 打印其中一个节点的嵌入向量
# print(w2v_model.wv.key_to_index)

w2v_model.wv.save_word2vec_format(generate+'emb.vector', binary=False)
w2v_model.save(generate+'emb.model')
print(w2v_model.wv.most_similar('北平市',topn=10))
print("??")
print(w2v_model.wv.most_similar('北平市',topn=100))

 28%|██▊       | 22/80 [05:23<14:11, 14.69s/it]


KeyboardInterrupt: 

In [4]:
from gensim.models import Word2Vec,word2vec,KeyedVectors
import numpy as np
from tqdm import tqdm
import pandas as pd

generate='./data/generate/'
path='./data/rec_data/'
model=KeyedVectors.load_word2vec_format(generate+'emb.vector')

print(model['小米'])

# print(model.wv.most_similar('小米',topn=10))

# 对每个句子的所有词向量取均值，来生成一个句子的vector
def build_sentence_vector(sentence, size, w2v_model):
    sen_vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in sentence:
        try:
            sen_vec += w2v_model[word.replace(" ","")].reshape((1, size))
            count += 1
        except KeyError:
            print("ygq:",word)
            continue
    if count != 0:
        sen_vec /= count
    return sen_vec

ids=[]
entitys=[]
with open(path+'recommend_content_entity_0317.txt','r') as fileTrainRaw:
    for line in tqdm(fileTrainRaw):
        tmps=eval(line)
        id=tmps.get('id')
        entity=tmps.get('entity')
        emb=build_sentence_vector(entity,64,model)
        ids.append(id)
        entitys.append(emb.tolist()[0])
with open(path+'recommend_content_entity_train.txt','r') as fileTrainRaw:
    for line in tqdm(fileTrainRaw):
        tmps=eval(line)
        id=tmps.get('id')
        entity=tmps.get('entity')
        emb=build_sentence_vector(entity,64,model)
        ids.append(id)
        entitys.append(emb.tolist()[0])
with open(path+'recommend_content_entity_test.txt','r') as fileTrainRaw:
    for line in tqdm(fileTrainRaw):
        tmps=eval(line)
        id=tmps.get('id')
        entity=tmps.get('entity')
        emb=build_sentence_vector(entity,64,model)
        ids.append(id)
        entitys.append(emb.tolist()[0])

cols={
  'id':ids,
  'entity':entitys
}
data=pd.DataFrame(cols)
data.to_feather(generate+'entityEmbedding.feather')
# data.to_csv(generate+'entityEmbedding.txt',sep='\t',index=False,header=False)
# # 存在一个问题，向量存进去变成了str

[ 0.03990203  0.0641895  -0.34885606  0.27093658 -0.22365998  0.3471349
  0.15444866  0.3447451  -0.21447933  0.4822503  -0.26895115 -0.1906698
 -0.53301394 -0.49476147  0.01848119  0.17852516 -0.14472598  0.35592097
  0.11939192 -0.04230782  0.27578312  0.2562184  -0.23706503  0.22594729
  0.12283963  0.20185483 -0.16958737  0.00133557  0.3395754   0.05694688
 -0.01897004  0.39143223 -0.32678825  0.19225955 -0.29233202  0.13874243
 -0.40654367 -0.06401161  0.54609334 -0.03138198  0.31439874 -0.02534526
  0.25911382 -0.16106983  0.3413832  -0.25512263  0.21221015  0.21619022
 -0.20629704 -0.08702236  0.16214317  0.20242283 -0.12982441 -0.42438692
 -0.11670694  0.5105656   0.07102436  0.00733334  0.31607357 -0.31519628
  0.04184337  0.4215892   0.19965126  0.27427056]


610it [00:00, 6091.73it/s]

ygq: 交流充电枪
ygq: Raw格式
ygq: 打卡指纹膜
ygq: 玉韫珠藏
ygq: 金蝶KIS记账王
ygq: 柏融
ygq: 小世君
ygq: 老荣华
ygq: 循环流化床
ygq: 小嗨妹
ygq: 圆号
ygq: 奴隶社会
ygq: 荣信自然
ygq: 胜利镇
ygq: hifi
ygq: 呆呆华
ygq: 松脱封
ygq: 粉末冶金工艺
ygq: 中国养兔联盟
ygq: 无忧然
ygq: 莱特莱德


1220it [00:00, 6054.16it/s]

ygq: 熊猫纪念币
ygq: 广州市会场家电子商务有限公司


...

1446it [00:00, 7990.42it/s]


ygq: 新意互动


803it [00:00, 8029.43it/s]

ygq: 知世


1535it [00:00, 8120.64it/s]


In [ ]:
data

1. 上述已经生成文章向量表示
2. 下面只是对文件进行略微调整

In [ ]:
entity = data['entity'].apply(lambda x: pd.Series(x))
entity.columns = ['embedding_{}'.format(i) for i in range(64)]
data = pd.concat([data, entity], axis=1)

data.rename(columns={'id':'itemId'},inplace=True)
del data['entity']

In [ ]:
data.to_feather(generate+'entityEmbedding.feather')

In [ ]:
# 读取数据，content和entity
itemId=[]
entity=[]
with open(path+'recommend_content_entity_0317.txt','r') as fileTrainRaw:
  for line in tqdm(fileTrainRaw):
    tmps=eval(line)
    id=tmps.get('id')
    tmps=tmps.get('entity')
    n=len(tmps)
    for i in range(n):
        itemId.append(id)
        entity.append(tmps[i].replace(" ",""))
with open(path+'recommend_content_entity_train.txt','r') as fileTrainRaw:
  for line in tqdm(fileTrainRaw):
    tmps=eval(line)
    id=tmps.get('id')
    tmps=tmps.get('entity')
    n=len(tmps)
    for i in range(n):
        itemId.append(id)
        entity.append(tmps[i].replace(" ",""))
with open(path+'recommend_content_entity_test.txt','r') as fileTrainRaw:
  for line in tqdm(fileTrainRaw):
    tmps=eval(line)
    id=tmps.get('id')
    tmps=tmps.get('entity')
    n=len(tmps)
    for i in range(n):
        itemId.append(id)
        entity.append(tmps[i].replace(" ",""))

In [ ]:
cols={
  'itemId':itemId,
  'entity':entity
}
data=pd.DataFrame(cols)
data.to_feather(generate+'entitys_raw.feather')

In [2]:
# 导入包
import pandas as pd
from tqdm import tqdm

generate='./data/generate/'
path='./data/rec_data/'

In [5]:
data=pd.read_feather(generate+'entityEmbedding.feather')

In [7]:
data.drop_duplicates()

,itemId,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,...,embedding_54,embedding_55,embedding_56,embedding_57,embedding_58,embedding_59,embedding_60,embedding_61,embedding_62,embedding_63
0,10000000,0.169619,0.067012,-0.184172,-0.511508,0.183153,-0.095429,0.083347,-0.077358,-0.543327,...,0.159309,0.712660,0.100601,0.949217,0.793156,0.416812,-0.331006,0.128246,0.097909,0.135928
1,10000001,0.144607,-0.779346,0.060994,0.015341,0.199975,-0.671673,0.566447,-0.284230,0.261507,...,0.415479,0.588928,-0.076894,0.130692,0.252781,0.359660,0.840597,0.510573,0.635659,-0.490312
2,10000002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,10000003,-0.016664,0.348830,-0.783774,-0.134493,0.004738,0.022774,-0.507214,0.116409,0.402529,...,-0.015087,0.705131,0.207758,-0.421699,-0.335046,-0.109649,-0.314808,0.391750,0.324040,0.250994
4,10000004,0.121303,-0.054179,-0.323527,0.121021,0.028779,-0.030692,-0.137862,-0.439544,0.347013,...,0.228866,0.245712,-0.372444,0.189387,0.117173,-0.351487,-0.311415,-0.024203,0.743892,0.199688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146786,10146786,0.471591,0.157233,-0.332175,0.254731,-0.207703,0.052986,0.133995,-0.014011,0.025309,...,0.520480,0.128606,0.017385,0.022702,-0.094646,-0.237642,-0.246007,-0.030711,0.145421,0.087445
146787,10146787,0.338299,-0.063012,-0.134737,-0.023718,-0.488543,-0.292943,0.546237,0.203084,-0.100674,...,-0.301319,0.380286,0.234763,0.014832,0.508225,-0.187779,-0.457506,0.082441,-0.083602,-0.182759
146788,10146788,0.052775,0.254540,0.220045,0.173306,0.072730,0.075440,0.076344,0.303689,-0.308996,...,0.433590,0.705914,-0.326444,0.086536,0.021641,-0.039090,-0.396556,0.031085,-0.113053,0.134469
146789,10146789,2.018363,-0.255575,-2.160951,-1.083886,0.219248,-1.659263,-0.972981,0.766477,-0.941354,...,-0.379994,0.197484,-1.136956,0.230440,-0.021388,-0.605519,-0.397280,0.439148,1.145469,-0.601148
